In [ ]:
import pandas as pd
import altair as alt
import numpy as np
import statsmodels.api as sm
from scipy import stats
import re

In [ ]:
input_files = '../Data/filtered_ppj_data/pillar_project_data_inputs.xlsx'
carriers = '/Users/ivan/Downloads/7_genes_for_Starita_02282025_hg38.xlsx' #carriers data
bridges_all = '../Data/BRIDGES_data/20250815_BRIDGES_missense_all.xlsx' #bridges all missense variants
bridges_population = '../Data/BRIDGES_data/20250815_BRIDGES_missense_population.xlsx' #bridges population missense variants
bridges_all_ptv = '../Data/BRIDGES_data/20250815_BRIDGES_PTVs_all.xlsx' #bridges all PTVs
bridges_pop_ptv = '../Data/BRIDGES_data/20250815_BRIDGES_PTVs_pop.xlsx' #bridges population PTVs

In [ ]:
def read_inputs(input_files, carriers, bridges_all_vars, bridges_population, bridges_all_ptv, bridges_pop_ptv):
    sge_inputs = pd.read_excel(input_files)
    sge_genes = list(set(sge_inputs['gene'].tolist()))

    carriers_df = pd.read_excel(carriers)
    carriers_df = carriers_df.loc[carriers_df['CAVA_GENE'].isin(sge_genes)]
    
    bridges_all = pd.ExcelFile(bridges_all_vars)
    
    bridges_genes = bridges.sheet_names

    bridges_pop = pd.ExcelFile(bridges_population)
    bridges_allptv = pd.ExcelFile(bridges_all_ptv)
    bridges_popptv = pd.Excelfile(bridges_pop_ptv)


    return sge_inputs, sge_genes, carriers_df, bridges_genes, bridges_all, bridges_pop, bridges_allptv, bridges_popptv

In [ ]:
def main():
    sge_input, sge_genes, carriers, bridges_genes, bridges_all, bridges_pop, bridges_allptv, bridges_popptv =read_inputs(input_files, carriers, bridges_all)

In [ ]:
main()